In [46]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamgamelog, leaguegamefinder, leaguegamelog, commonteamroster, teamdashlineups
from nba_api.live.nba.endpoints import scoreboard, playbyplay, boxscore
from nba_api.stats.library.parameters import SeasonAll
from datetime import datetime, timezone
from dateutil import parser
import pandas as pd
from pprint import pprint
from IPython.display import display
import requests
import time

In [47]:
# Get the list of all NBA teams
nba_teams = teams.get_teams()
teams_df = pd.DataFrame(nba_teams)
teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [60]:
# Specify the team name you're interested in analyzing
team_abbreviation = input("Enter the team to predict against: ")

Enter the team to predict against:  PHI


In [67]:
# ROSTERS

team_id = teams_df.loc[teams_df['abbreviation'] == team_abbreviation, 'id'].iloc[0]
team_id = get_team_id(team_abbreviation)
print(f"The team ID for {team_abbreviation} is {team_id}.")
teamfinder = commonteamroster.CommonTeamRoster(team_id=team_id, league_id_nullable='00')
teams = teamfinder.get_data_frames()[0]
team_id = teams['TeamID'].unique().tolist()
team_ids = teams['TeamID'].unique().tolist()
teams_roster=[]
for team_id in team_ids:
    player_stats_data = commonteamroster.CommonTeamRoster(season='2023-24', team_id=team_id)   # <--- ADD YOUR season parameter!
    df = player_stats_data.common_team_roster.get_data_frame()
    df['TeamID'] = team_id
    teams_roster.append(df)
    
df

The team ID for PHI is 1610612755.


,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612755,2023,00,Tyrese Maxey,Tyrese,tyrese-maxey,0,G,6-2,200,"NOV 04, 2000",23.0,3,Kentucky,1630178,#21 Pick in 2020 Draft
1,1610612755,2023,00,KJ Martin,KJ,kj-martin,1,F,6-6,215,"JAN 06, 2001",22.0,3,IMG Academy (FL),1630231,Traded from LAC on 11/01/23
2,1610612755,2023,00,Marcus Morris Sr.,Marcus,marcus-morris-sr,5,F,6-8,218,"SEP 02, 1989",34.0,12,Kansas,202694,Traded from LAC on 11/01/23
3,1610612755,2023,00,Mo Bamba,Mo,mo-bamba,7,C,7-0,231,"MAY 12, 1998",25.0,5,Texas,1628964,Signed on 07/09/23
4,1610612755,2023,00,De'Anthony Melton,De'Anthony,deanthony-melton,8,G,6-3,200,"MAY 28, 1998",25.0,5,Southern California,1629001,Traded from MEM on 06/24/22
5,1610612755,2023,00,Kelly Oubre Jr.,Kelly,kelly-oubre-jr,9,F-G,6-7,203,"DEC 09, 1995",27.0,8,Kansas,1626162,Signed on 09/26/23
6,1610612755,2023,00,Jaden Springer,Jaden,jaden-springer,11,G,6-4,202,"SEP 25, 2002",21.0,2,Tennessee,1630531,#28 Pick in 2021 Draft
7,1610612755,2023,00,Tobias Harris,Tobias,tobias-harris,12,F,6-8,226,"JUL 15, 1992",31.0,12,Tennessee,202699,Traded from LAC on 02/06/19
8,1610612755,2023,00,Ricky Council IV,Ricky,ricky-council-iv,16,G,6-6,207,"AUG 03, 2001",22.0,R,Arkansas,1641741,Signed on 07/01/23
9,1610612755,2023,00,Joel Embiid,Joel,joel-embiid,21,C-F,7-0,280,"MAR 16, 1994",29.0,7,Kansas,203954,#3 Pick in 2014 Draft


In [69]:
# TEAM INFO
team_info = [team for team in nba_teams if team["abbreviation"] == team_abbreviation][0]
team_info

{'id': 1610612755,
 'full_name': 'Philadelphia 76ers',
 'abbreviation': 'PHI',
 'nickname': '76ers',
 'city': 'Philadelphia',
 'state': 'Pennsylvania',
 'year_founded': 1949}

In [78]:
# TEAMS GAME LOG SINCE 1983
# can be used to calculate franchise wins/losses etc
# can also break it down by season_id to get gamelog just for this szn

# Get the team ID
team_id = teams_df.loc[teams_df['abbreviation'] == team_abbreviation, 'id'].iloc[0]

# Fetch the game log for the team using the obtained team ID
ourTeam_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id).get_data_frames()[0]
ourTeam_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612755,PHI,Philadelphia 76ers,0022300040,2023-11-21,PHI vs. CLE,L,265,119,...,0.824,14,30,44,19,12.0,8,14,25,-3.0
1,22023,1610612755,PHI,Philadelphia 76ers,0022300208,2023-11-19,PHI @ BKN,W,239,121,...,0.944,10,34,44,32,8.0,4,4,18,22.0
2,22023,1610612755,PHI,Philadelphia 76ers,0022300029,2023-11-17,PHI @ ATL,W,238,126,...,0.964,13,30,43,32,9.0,4,15,20,10.0
3,22023,1610612755,PHI,Philadelphia 76ers,0022300194,2023-11-15,PHI vs. BOS,L,240,107,...,0.818,10,31,41,18,5.0,1,9,14,-10.0
4,22023,1610612755,PHI,Philadelphia 76ers,0022300019,2023-11-14,PHI vs. IND,L,241,126,...,0.971,5,33,38,22,5.0,3,16,26,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,21983,1610612755,PHL,Philadelphia 76ers,0028300050,1983-11-05,PHL @ NJN,W,240,119,...,0.730,19,26,45,24,12.0,9,20,25,NaN
3642,21983,1610612755,PHL,Philadelphia 76ers,0028300042,1983-11-04,PHL vs. MIL,L,240,94,...,0.815,17,27,44,21,2.0,4,21,27,NaN
3643,21983,1610612755,PHL,Philadelphia 76ers,0028300025,1983-11-01,PHL vs. CHI,W,240,99,...,0.700,12,39,51,27,9.0,19,22,19,NaN
3644,21983,1610612755,PHL,Philadelphia 76ers,0028300017,1983-10-29,PHL @ IND,W,265,124,...,0.745,18,31,49,27,14.0,5,27,32,NaN


In [18]:
ourTeam_games = pd.DataFrame(ourTeam_games)

# Count the number of wins and losses
total_wins = ourTeam_games[ourTeam_games['WL'] == 'W'].shape[0]
total_losses = ourTeam_games[ourTeam_games['WL'] == 'L'].shape[0]

# Display the overall record
print(f'{team_abb} since 1983: Won - {total_wins} | Loss - {total_losses}')

PHI since 1983: Won - 1738 | Loss - 1906


In [15]:
#teams = teams.get_teams()
opp = [x for x in teams if x['abbreviation'] == opp_abb][0]
opp_id = opp['id']

#this time we convert it to a dataframe in the same line of code
opp_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=opp_id).get_data_frames()[0]
opp_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612747,LAL,Los Angeles Lakers,0022300216,2023-11-19,LAL vs. HOU,W,240,105,...,0.724,10,32,42,26,13,1,17,13,1.0
1,22023,1610612747,LAL,Los Angeles Lakers,0022300036,2023-11-17,LAL @ POR,W,241,107,...,0.682,11,33,44,27,11,6,18,16,13.0
2,22023,1610612747,LAL,Los Angeles Lakers,0022300198,2023-11-15,LAL vs. SAC,L,239,110,...,0.789,7,36,43,25,12,6,23,17,-15.0
3,22023,1610612747,LAL,Los Angeles Lakers,0022300026,2023-11-14,LAL vs. MEM,W,241,134,...,0.750,10,44,54,30,4,10,18,13,27.0
4,22023,1610612747,LAL,Los Angeles Lakers,0022300187,2023-11-12,LAL vs. POR,W,240,116,...,0.811,7,34,41,27,8,4,9,11,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,21983,1610612747,LAL,Los Angeles Lakers,0028300066,1983-11-08,LAL @ DEN,W,240,133,...,0.917,22,31,53,30,11,8,23,31,NaN
3881,21983,1610612747,LAL,Los Angeles Lakers,0028300053,1983-11-05,LAL @ DAL,L,240,102,...,0.667,18,23,41,29,7,8,20,27,NaN
3882,21983,1610612747,LAL,Los Angeles Lakers,0028300035,1983-11-02,LAL @ SDC,L,240,106,...,0.760,16,33,49,29,14,7,24,24,NaN
3883,21983,1610612747,LAL,Los Angeles Lakers,0028300010,1983-10-29,LAL @ UTH,W,240,120,...,0.718,29,31,60,25,10,10,20,41,NaN


In [19]:
opp_games = pd.DataFrame(opp_games)

# Count the number of wins and losses
total_wins = opp_games[opp_games['WL'] == 'W'].shape[0]
total_losses = opp_games[opp_games['WL'] == 'L'].shape[0]

# Display the overall record
print(f'{opp_abb} since 1983: Won - {total_wins} | Loss - {total_losses}')

LAL since 1983: Won - 2288 | Loss - 1597


In [37]:
# Define a mock dictionary
mock_dict = {
    "PG": {
        "Starter": ["Tyrese Maxey"],
        "Bench 1": ["Pat Bev"],
        "Bench 2": ["Jaden Springer"]
    },
    "SG": {
        "Starter": ["Kelly Oubre"],
        "Bench 1": ["Deanthony Melton"],
        "Bench 2": ["Furkan Korkmaz"]
    },
    "SF": {
        "Starter": ["Robert Covington"],
        "Bench 1": ["Danuel House JrS"],
        "Bench 2": ["Jaden Springer"]
    },
    "PF": {
        "Starter": ["Tobias Harris"],
        "Bench 1": ["Marcus Morris"],
        "Bench 2": ["Jaden Springer"]
    },
    "C": {
        "Starter": ["Joel Embiid"],
        "Bench 1": ["Paul Reed"],
        "Bench 2": ["Mo Bamba"]
    }
}

# Print the mock dictionary
mock_df = pd.DataFrame(mock_dict)
mock_df

,PG,SG,SF,PF,C
Starter,[Tyrese Maxey],[Kelly Oubre],[Robert Covington],[Tobias Harris],[Joel Embiid]
Bench 1,[Pat Bev],[Deanthony Melton],[Danuel House JrS],[Marcus Morris],[Paul Reed]
Bench 2,[Jaden Springer],[Furkan Korkmaz],[Jaden Springer],[Jaden Springer],[Mo Bamba]


In [22]:
lineup = teamdashlineups.TeamDashLineups(team_id='1610612744').get_data_frames()[0]
lineup


,GROUP_SET,GROUP_VALUE,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GP,W,L,W_PCT,MIN,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Overall,2023-24,1610612744,GSW,Golden State Warriors,14,6,8,0.429,677.0,...,1,1,1,1,1,1,1,1,1,1


In [24]:
from nba_api.stats.endpoints import teamplayerdashboard
import pandas as pd

# Replace '1610612744' with the actual team ID
team_id = '1610612744'

# Get team lineup data
lineup = teamplayerdashboard.TeamPlayerDashboard(team_id=team_id).get_data_frames()[0]

# Display the lineup DataFrame
lineup

,GROUP_SET,TEAM_ID,TEAM_NAME,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Overall,1610612744,Golden State Warriors,2023-24,14,6,8,0.429,677.0,570,...,1,1,1,1,1,1,1,1,1,1
